In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms 
import torchvision.transforms as T
from torchvision.models import resnet

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class GermanTraffic(Dataset):
    def __init__(self, root_dir, transform=None, train=True, verify_ratio=0.2):
        self.root_dir = root_dir
        self.transform = transform
        self.categories = os.listdir(root_dir)
        self.category_mapping = {category: int(category) for idx, category in enumerate(self.categories)}
        self.filepaths = self._get_filepaths()

        if train:
            self.filepaths = self._split_data(verify_ratio)[0]
        else:
            self.filepaths = self._split_data(verify_ratio)[1]

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        filepath, label = self.filepaths[idx]
        image = Image.open(filepath).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _get_filepaths(self):
        filepaths = []
        for category in self.categories:
            category_folder = os.path.join(self.root_dir, category)
            image_files = os.listdir(category_folder)
            for image_file in image_files:
                image_path = os.path.join(category_folder, image_file)
                filepaths.append((image_path, self.category_mapping[category]))

        return filepaths

    def _split_data(self, verify_ratio):
        filepaths_train, filepaths_verify = train_test_split(self.filepaths, test_size=verify_ratio)
        return filepaths_train, filepaths_verify


In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_ds = GermanTraffic('../input/acc-german-traffic-sign-classification/GTSRB_Challenge/train/',transform=transform)
valid_ds = GermanTraffic('../input/acc-german-traffic-sign-classification/GTSRB_Challenge/train/',transform=transform,train=False)

In [4]:
train_dl = DataLoader(train_ds,batch_size=64)
valid_dl = DataLoader(valid_ds,batch_size=64)

In [5]:
model = resnet.resnet34(num_classes=43)

In [6]:
device = torch.device('cuda:0')
model = model.to(device)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim


# Define your loss function
criterion = nn.CrossEntropyLoss()

# Define your optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_dl:
        optimizer.zero_grad()
        inputs=inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_dl)

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valid_dl:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_val_loss = running_val_loss / len(valid_dl)
    val_accuracy = correct / total

    # Print average loss and accuracy for the epoch
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f} | Val Accuracy = {val_accuracy:.2%}")



Epoch 1: Train Loss = 2.9460 | Val Loss = 2.4802 | Val Accuracy = 30.59%
Epoch 2: Train Loss = 1.8178 | Val Loss = 1.6202 | Val Accuracy = 50.94%
Epoch 3: Train Loss = 1.0229 | Val Loss = 0.9571 | Val Accuracy = 73.53%
Epoch 4: Train Loss = 0.4542 | Val Loss = 0.2982 | Val Accuracy = 92.59%
Epoch 5: Train Loss = 0.1949 | Val Loss = 0.2397 | Val Accuracy = 94.04%
Epoch 6: Train Loss = 0.0912 | Val Loss = 0.0696 | Val Accuracy = 98.89%
Epoch 7: Train Loss = 0.0425 | Val Loss = 0.0749 | Val Accuracy = 98.37%
Epoch 8: Train Loss = 0.0226 | Val Loss = 0.0194 | Val Accuracy = 99.72%
Epoch 9: Train Loss = 0.0128 | Val Loss = 0.0121 | Val Accuracy = 99.80%
Epoch 10: Train Loss = 0.0087 | Val Loss = 0.0096 | Val Accuracy = 99.82%


In [8]:
save_path = "gts_state_dict.pth"

# Save only the model's state_dict (learned parameters)
torch.save(model.state_dict(), save_path)